# Deep Learning Assignment

## Feed forward network using MNIST Classification

In [1]:
import os
from pathlib import Path
import torch
from torch.utils.data import TensorDataset ,DataLoader
from torch import nn,optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
PATH=Path("C:\\Users\\Sree Soundarya\\Desktop\\Deep Learning")

In [2]:
train=pd.read_csv(PATH/'train.csv')
test=pd.read_csv(PATH/'test.csv')
x=train.drop("label",axis=1)
y=np.array(train['label'])

torch_X_train = torch.from_numpy(x.values).type(torch.FloatTensor)/255
torch_y_train = torch.from_numpy(y).type(torch.LongTensor)
myDataset = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
valid_no  = int(0.2 * len(myDataset))
trainSet,testSet = torch.utils.data.random_split(myDataset,(len(myDataset)-valid_no,valid_no))
print(f"len of trainSet {len(trainSet)} , len of testSet {len(testSet)}")
batch_size=50
train_loader  = DataLoader(trainSet , batch_size=batch_size ,shuffle=True) 
test_loader  = DataLoader(testSet , batch_size=batch_size ,shuffle=True)

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 100)
        self.fc2 = nn.Linear(100, 175)
        self.fc3 = nn.Linear(175, 10)
        # Dropout module with 0.2 drop probability
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = F.softmax(self.fc3(x), dim=1)
        return x
        
model=Network()
optimizer=optim.SGD(model.parameters(),lr=0.01)
criterion=nn.NLLLoss()
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)
list=[7,10]
for i in list:
    epochs=i
    train_losses,test_losses=[],[]
    for e in range(epochs):
        running_loss=0
        for images,labels in train_loader:
            optimizer.zero_grad()
            ps=model(images)
            loss=criterion(ps,labels)
            loss.backward()
            optimizer.step()
            running_loss+=loss.item()

        else:
            test_loss=0
            accuracy=0

            with torch.no_grad():
                model.eval()
                for images,labels in test_loader:
                    ps=model(images)
                    test_loss+=criterion(ps,labels)
                    ps=torch.exp(ps)
                    top_p,top_class=ps.topk(1,dim=1)
                    equals=top_class==labels.view(*top_class.shape)
                    accuracy+=torch.mean(equals.type(torch.FloatTensor))
            model.train()
            train_losses.append(running_loss/len(train_loader))
            test_losses.append(test_loss/len(test_loader))

            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
                  "Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
                  "Test Accuracy: {:.3f}".format(accuracy/len(test_loader)))

len of trainSet 33600 , len of testSet 8400
97935
Epoch: 1/7..  Training Loss: -0.101..  Test Loss: -0.102..  Test Accuracy: 0.116
Epoch: 2/7..  Training Loss: -0.103..  Test Loss: -0.105..  Test Accuracy: 0.156
Epoch: 3/7..  Training Loss: -0.107..  Test Loss: -0.109..  Test Accuracy: 0.368
Epoch: 4/7..  Training Loss: -0.113..  Test Loss: -0.118..  Test Accuracy: 0.402
Epoch: 5/7..  Training Loss: -0.132..  Test Loss: -0.158..  Test Accuracy: 0.257
Epoch: 6/7..  Training Loss: -0.191..  Test Loss: -0.234..  Test Accuracy: 0.414
Epoch: 7/7..  Training Loss: -0.293..  Test Loss: -0.366..  Test Accuracy: 0.432
Epoch: 1/10..  Training Loss: -0.405..  Test Loss: -0.459..  Test Accuracy: 0.498
Epoch: 2/10..  Training Loss: -0.473..  Test Loss: -0.511..  Test Accuracy: 0.552
Epoch: 3/10..  Training Loss: -0.510..  Test Loss: -0.534..  Test Accuracy: 0.560
Epoch: 4/10..  Training Loss: -0.529..  Test Loss: -0.551..  Test Accuracy: 0.564
Epoch: 5/10..  Training Loss: -0.557..  Test Loss: -0.5